# Similarity Classification Using Sentence Pair Comparison with RoBERTa
We have generated the labelled dataset from the previous notebook create_label.ipynb.
<br>The datasets are split into training set and the validation set and put into variables train_df and eval_df respectively.

[simpletransformer](https://simpletransformers.ai/) is a wrapper to Huggingface's transformer library which helps to standardize many of the training tasks and has a nice integration with Wegihts and Bias (WandB) visualization.

The task of finding the candidate from the resume is essentially [GLUE](https://gluebenchmark.com/) task [Semantic Textual Similarity Benchmark (STS-B)]

In here I am using the **ClassificationModel** with _regression=**True**_ which is suitable for providing a single value, in this case the similarity of two sentences.

Instead of the original BERT for the pretrained model, I chose RoBERTa which uses the same engine but a different training method and more training time.
<br>It performs reasonably well even with a trimmed-down training/eval set to allow this runnable for the project submission.
<br>Comparatively, the original BERT base pretrained model (bert-base-cased) has a limited pretrained vocab and left lots of tokens marked as unknown \[UNK\]. Instead of using the BERT large I have picked the RoBERTa base (roberta-base) instead, which has a vocab of over 50k and covered most of the words used.

In [1]:
# simpletransformers only works with this version
!pip install transformers==3.0.2

In [2]:
# Read back the training and eval set we have created in create_label.ipynb.
import pandas as pd
suffix = "_submit"
train_df = pd.read_excel(f"../data/train_df{suffix}.xlsx")
eval_df = pd.read_excel(f"../data/eval_df{suffix}.xlsx")

In [3]:
train_df

,Unnamed: 0,title_job,text_a,title_res,text_b,labels
0,0,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074
1,4,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074
2,5,c/ c++ software engineer,Dom Daniel Armenia is looking for dynamic self...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074
3,6,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074
4,10,java/j2ee developer,PointSource is seeking full time J2EE Develope...,sr. java / j2ee developer,Responsibilities: \n• Involved in Requirements...,4.280726
...,...,...,...,...,...,...
3839,7611,flash/ as3 developer,NY based social and mobile games startup is ge...,developer & solution analyst,"Domain Banking \nTechnology MVC, C#.net, JQuer...",1.575380
3840,7613,unix systems administrator,Administration of Corporate Unix servers Plan...,senior developer/ business analyst,12-Sep-2012 Present Senior Developer/ Business...,0.273131
3841,7614,unix systems administrator,Administration of corporate Unix Solaris Linu...,senior developer/ business analyst,12-Sep-2012 Present Senior Developer/ Business...,0.273131
3842,7615,unix systems administrator,ArmenTel is seeking for candidates to fulfill ...,senior developer/ business analyst,12-Sep-2012 Present Senior Developer/ Business...,0.273131


In [4]:
eval_df.shape

(1914, 6)

In [5]:
!wandb login 20427a748e4feb47468643e4770a018a2152cc0c
import wandb
id = wandb.util.generate_id()
wandb.init(project='vec2rec-roberta', id=id, resume="allow")

Successfully logged in to Weights & Biases!


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\kerkermui/.netrc


W&B Run: https://app.wandb.ai/crispin/vec2rec-roberta/runs/2ub2w7hf

In [6]:
'''
Initialize the model. Points of note:
1) class ClassificationModel is selected with roberta-base as the pretrained model.
2) It will take in evaluation data to determine / save the best model under outputs/best_model, happens every 200 steps.
3) The model is trained on GPU. If run on Kaggle GPUs, it can achieve train_batch_size of 16. My home PC can only use 6.
4) The max_seq_length determines the max length of the sentence pairs, based on previous statistics calculated 512 is enough.
5) The epoch is set to 5 which is often enough for larger datasets. It will also use the evaluation data performance to
   determine whether it can be stopped if there are no additional drops in the loss function. Given the small value of epochs
   this is also set small to avoid it from stopping prematurely.
6) The learning rate not shown here takes the default of 4e-5. However the model will also automatically adjust lr by default.
7) WandB is configured to plot the progress of the training (shown below).
'''
from simpletransformers.classification import ClassificationModel
train_args = {
    'reprocess_input_data': True,
    'evaluate_during_training': True,
    'evaluate_during_training_steps': 200,
    'max_seq_length': 512,
    'num_train_epochs': 5,
    'train_batch_size': 6,
    #'train_batch_size': 16,
    'wandb_project': 'vec2rec-roberta',
    'wandb_kwargs':{"id":id, "resume":True},
    #'no-cache':True,
    #'no_save':True,
    #'save_model_every_epoch':False,
    'save_eval_checkpoints':False,
    'save_steps':False,
    #'best_model_dir':"/kaggle/tmp/outputs/best_model"
    #'output_dir':"/kaggle/tmp/outputs/",
    'overwrite_output_dir':True,
    'use_early_stopping':True,
    'early_stopping_delta':0.001,
    'early_stopping_consider_epochs':True,
    'regression': True,
}
model = ClassificationModel('roberta', 'roberta-base', num_labels=1, args=train_args, use_cuda=True)
model.args

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

ClassificationArgs(adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', custom_layer_parameters=[], custom_parameter_groups=[], train_custom_parameters_only=False, config={}, dataloader_num_workers=6, do_lower_case=False, early_stopping_consider_epochs=True, early_stopping_delta=0.001, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=True, evaluate_during_training_silent=True, evaluate_during_training_steps=200, evaluate_during_training_verbose=False, fp16=True, gradient_accumulation_steps=1, learning_rate=4e-05, local_rank=-1, logging_steps=50, manual_seed=None, max_grad_norm=1.0, max_seq_length=512, multiprocessing_chunksize=500, n_gpu=1, no_cache=False, no_save=False, num_train_epochs=5, output_dir='outputs/', overwrite_output_dir=True, process_count=6, reprocess_input_data=True, save_best_model=True, save_eval_checkpoints=False, save_model_every_e

In [7]:
# The Pearson and Spearman correlations are metrics used for STS-B
# https://gluebenchmark.com/tasks
from scipy.stats import pearsonr, spearmanr

def pearson_corr(preds, labels):
    return pearsonr(preds, labels)[0]

def spearman_corr(preds, labels):
    return spearmanr(preds, labels)[0]

In [8]:
import pytz
from datetime import datetime

print(datetime.now(pytz.timezone('Hongkong')))

2020-08-07 21:02:13.308315+08:00


In [9]:
%%time
%%wandb
# The model is trained and timed here with statistics shown in WandB websites displayed. The sample run took around 33 mins.
model.train_model(train_df, eval_df=eval_df, pearson_corr=pearson_corr, spearman_corr=spearman_corr)

D:\ProgramData\Anaconda3\envs\vec2rec-with-RoBERTa\lib\site-packages\torch\optim\lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
D:\ProgramData\Anaconda3\envs\vec2rec-with-RoBERTa\lib\site-packages\torch\optim\lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
D:\ProgramData\Anaconda3\envs\vec2rec-with-RoBERTa\lib\site-packages\torch\optim\lr_scheduler.py:200: UserWarning: Please also save or load the



Wall time: 33min 36s


In [10]:
# Get the final results on the eval dataset. Given this is not a discrete value classification, we can ignore the wrong_predictions
result, model_outputs, wrong_predictions = model.eval_model(eval_df, pearson_corr=pearson_corr, spearman_corr=spearman_corr)

In [11]:
# The final loss is 0.042 with both correlations higher than 94% which is quite good given the small dataset
result

{'pearson_corr': 0.9704873199597329,
 'spearman_corr': 0.9384291351353189,
 'eval_loss': 0.04246755579870296}

In [12]:
import pytz
from datetime import datetime

print(datetime.now(pytz.timezone('Hongkong')))

2020-08-07 21:36:34.327369+08:00


In [13]:
pd.DataFrame(model_outputs).describe()

,0
count,1914.000000
mean,2.197480
std,0.831456
min,0.101427
25%,2.189659
50%,2.401755
75%,2.632547
max,4.170128


In [17]:
# Here we calculate the per row error in the eval dataset into the compare column
compare_df = pd.concat([eval_df.loc[:, "title_job":"labels"], pd.DataFrame(model_outputs, index=eval_df.index, columns=["predict"])], axis=1)
compare_df["error"] = abs(compare_df.labels - compare_df.predict)
compare_df

,title_job,text_a,title_res,text_b,labels,predict,error
0,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074,4.170128,0.148946
1,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074,4.169966,0.149108
2,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,c++ developer,"Atlanta, USA Aug 2011 - Sep 2012 \nProject Nam...",4.108096,4.111215,0.003119
3,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,c++ developer,"Atlanta, USA Aug 2011 - Sep 2012 \nProject Nam...",4.108096,4.111654,0.003558
4,c/ c++ software engineer,Dom Daniel Armenia is looking for dynamic self...,c++ developer,"Environment: CPP, Linux, JavaScript, GIT , RTC...",4.108096,4.066742,0.041354
...,...,...,...,...,...,...,...
1909,java intern,Internship opportunities are currently availab...,associate software developer,• Worked in the Feature Development of SAP Ana...,0.387010,0.346939,0.040071
1910,java intern,Internship opportunities are currently availab...,associate software developer,LPU UMS is a smart home grown web-based ERP (E...,0.387010,0.349583,0.037427
1911,java intern,Internship opportunities are currently availab...,associate software developer,• Businesses to Riders Group Project– Django +...,0.387010,0.352929,0.034081
1912,graphic designer / customer service operator,Time to Print salon of operative printing is l...,senior developer/ business analyst,12-Sep-2012 Present Senior Developer/ Business...,1.577018,0.899014,0.678004


In [18]:
# The eval loss above is essentially the mean_squared_error, while we can also calculate the mean_absolute error
# In this case we can see that for all of the eval set on average we have an error of 0.064 on a 5 point scale, which is not bad.
from sklearn.metrics import mean_absolute_error, mean_squared_error

print(mean_squared_error(compare_df.labels, compare_df.predict))
print(mean_absolute_error(compare_df.labels, compare_df.predict))

0.041848985454064035
0.07132642155234832


In [19]:
compare_df.error.describe()

count    1914.000000
mean        0.071326
std         0.191783
min         0.000004
25%         0.008940
50%         0.018791
75%         0.040119
max         2.470064
Name: error, dtype: float64

In [20]:
# Only 70 samples have a score error > 0.5
pd.cut(compare_df.error, bins=[-1,0.5,1,2,3,4,6], right=True).value_counts()

(-1.0, 0.5]    1856
(0.5, 1.0]       34
(1.0, 2.0]       22
(2.0, 3.0]        2
(4.0, 6.0]        0
(3.0, 4.0]        0
Name: error, dtype: int64

In [21]:
compare_df[compare_df.error < 0.5]

,title_job,text_a,title_res,text_b,labels,predict,error
0,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074,4.170128,0.148946
1,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074,4.169966,0.149108
2,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,c++ developer,"Atlanta, USA Aug 2011 - Sep 2012 \nProject Nam...",4.108096,4.111215,0.003119
3,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,c++ developer,"Atlanta, USA Aug 2011 - Sep 2012 \nProject Nam...",4.108096,4.111654,0.003558
4,c/ c++ software engineer,Dom Daniel Armenia is looking for dynamic self...,c++ developer,"Environment: CPP, Linux, JavaScript, GIT , RTC...",4.108096,4.066742,0.041354
...,...,...,...,...,...,...,...
1908,java intern,Internship opportunities are currently availab...,associate software developer,Worked on building Andhra Pradesh district por...,0.387010,0.341872,0.045138
1909,java intern,Internship opportunities are currently availab...,associate software developer,• Worked in the Feature Development of SAP Ana...,0.387010,0.346939,0.040071
1910,java intern,Internship opportunities are currently availab...,associate software developer,LPU UMS is a smart home grown web-based ERP (E...,0.387010,0.349583,0.037427
1911,java intern,Internship opportunities are currently availab...,associate software developer,• Businesses to Riders Group Project– Django +...,0.387010,0.352929,0.034081


In [22]:
pd.options.display.max_colwidth = None
print(compare_df[compare_df.error >= 1].shape)
compare_df[compare_df.error >= 1]

(24, 7)


,title_job,text_a,title_res,text_b,labels,predict,error
7,senior software developer/ technical leader,As Senior Software Developer Technical Leader the incumbent has to lead and work with developers and participate in the design and development projects he has to work with Architecture team in the organization understand product roadmaps and assume key role in the development of these products BS MS in Computer Science Information Services or related field Minimum of 10 year extensive development experience Experience as Technical Leader on in Linux OS with in depth knowledge of software development processes Deep technical and professional skills readiness to learn new technologies Extensive programming experience with standard management and instrumentation API Ability to work independently as well as with other members in the group preferably in leadership role Ability to write technical and functional specifications and present them to the other team members and occasionally to the management thorough understanding of software development processes and methodologies and knowledge of complete product development life cycle Strong inter personal skills ability to work both independently and as part of global team and thrive on technical challenges Excellent communication and writing skills and experience in writing product specifications and technical documentation Good communication skills in English language Enthusiastic and self motivated Good team player,"technical lead, developer","for 2013 aimed at Life and Pensions products, intended to bring about changes in projection rates, to be effective from January, 2014. \n \nRole Technical lead, Developer \nResponsibilities \n- Writing Technical Specification document using Class Diagrams and Sequence Diagrams \n \n- Technical lead to guide developers through the development, unit and integration testing, UAT phases of the project \n \n- Review and upgrade of code developed by team members \n \n- Coordinating with onsite coordinator in project testing and project implementation phases \n \n- Ensuring quality of deliverables to be compliant with AEGON standard and Integrated Quality Management System standard of Tata Consultancy Services, through Internal and External Quality Assurance checks \n \nSolution Environment Java/J2EE (Spring framework v2.5.6), SQL, Rational Application Developer, Rational Clearcase, TOAD",4.100830,2.552578,1.548252
8,senior software developer/ technical leader,As Senior Software Developer Technical Leader the incumbent has to lead and work with developers and participate in the design and development projects he has to work with Architecture team in the organization understand product roadmaps and assume key role in the development of these products BS MS in Computer Science Information Services or related field Minimum of 10 year extensive development experience Experience as Technical Leader on in Linux OS with in depth knowledge of software development processes Deep technical and professional skills readiness to learn new technologies Extensive programming experience with standard management and instrumentation API Ability to work independently as well as with other members in the group preferably in leadership role Ability to write technical and functional specifications and present them to the other team members and occasionally to the management thorough understanding of software development processes and methodologies and knowledge of complete product development life cycle Strong inter personal skills ability to work both independently and as part of global team and thrive on technical challenges Excellent communication and writing skills and experience in writing product specifications and technical documentation Good communication skills in English language Enthusiastic and self motivated Good team player,"technical lead, developer","Kolkata, India \nCustomer Grameen Phone, Bangladesh \nPeriod December 2014 - March 2016 \nDescription \nGr